In [ ]:
channel_types = ['eeg', 'eeg', 'eog', 'misc', 'emg', 'misc','misc']


In [1]:
import os
import mne
import xmltodict
import json

In [2]:
file_path = 'Z:/shhs/polysomnography/edfs/shhs1/'
files_psg = [file for file in os.listdir(file_path)]

In [3]:
files_psg.sort()
file = files_psg[0]
file

'shhs1-200001.edf'

In [4]:
data = mne.io.read_raw_edf(file_path + files_psg[0])

Extracting EDF parameters from Z:\shhs\polysomnography\edfs\shhs1\shhs1-200001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [5]:
raw_data = data.get_data()

In [6]:
channel_names = data.ch_names

In [7]:
channel_names

['SaO2',
 'H.R.',
 'EEG(sec)',
 'ECG',
 'EMG',
 'EOG(L)',
 'EOG(R)',
 'EEG',
 'THOR RES',
 'ABDO RES',
 'POSITION',
 'LIGHT',
 'NEW AIR',
 'OX stat']

In [8]:
# get sleep stages --> see Word
file_path_hyp = 'Z:/shhs/polysomnography/annotations-events-nsrr/shhs1/shhs1-200001-nsrr.xml'

with open(file_path_hyp) as fd:
    doc = xmltodict.parse(fd.read())

In [11]:
doc

OrderedDict([('PSGAnnotation',
              OrderedDict([('SoftwareVersion', 'Compumedics'),
                           ('EpochLength', '30'),
                           ('ScoredEvents',
                            OrderedDict([('ScoredEvent',
                                          [OrderedDict([('EventType', None),
                                                        ('EventConcept',
                                                         'Recording Start Time'),
                                                        ('Start', '0'),
                                                        ('Duration',
                                                         '32520.0'),
                                                        ('ClockTime',
                                                         '00.00.00 22.00.00')]),
                                           OrderedDict([('EventType',
                                                         'Respiratory|Respiratory'),
       

In [14]:
annotations = doc['PSGAnnotation']
annotations

OrderedDict([('SoftwareVersion', 'Compumedics'),
             ('EpochLength', '30'),
             ('ScoredEvents',
              OrderedDict([('ScoredEvent',
                            [OrderedDict([('EventType', None),
                                          ('EventConcept',
                                           'Recording Start Time'),
                                          ('Start', '0'),
                                          ('Duration', '32520.0'),
                                          ('ClockTime', '00.00.00 22.00.00')]),
                             OrderedDict([('EventType',
                                           'Respiratory|Respiratory'),
                                          ('EventConcept',
                                           'SpO2 artifact|SpO2 artifact'),
                                          ('Start', '304'),
                                          ('Duration', '3'),
                                          ('SignalLocation', 'SaO

In [16]:
events = annotations['ScoredEvents']
events

OrderedDict([('ScoredEvent',
              [OrderedDict([('EventType', None),
                            ('EventConcept', 'Recording Start Time'),
                            ('Start', '0'),
                            ('Duration', '32520.0'),
                            ('ClockTime', '00.00.00 22.00.00')]),
               OrderedDict([('EventType', 'Respiratory|Respiratory'),
                            ('EventConcept', 'SpO2 artifact|SpO2 artifact'),
                            ('Start', '304'),
                            ('Duration', '3'),
                            ('SignalLocation', 'SaO2')]),
               OrderedDict([('EventType', 'Arousals|Arousals'),
                            ('EventConcept', 'Arousal|Arousal ()'),
                            ('Start', '964.8'),
                            ('Duration', '25.4'),
                            ('SignalLocation', 'EMG')]),
               OrderedDict([('EventType', 'Arousals|Arousals'),
                            ('EventConce

In [18]:
scored_events = events['ScoredEvent']
scored_events  #list

[OrderedDict([('EventType', None),
              ('EventConcept', 'Recording Start Time'),
              ('Start', '0'),
              ('Duration', '32520.0'),
              ('ClockTime', '00.00.00 22.00.00')]),
 OrderedDict([('EventType', 'Respiratory|Respiratory'),
              ('EventConcept', 'SpO2 artifact|SpO2 artifact'),
              ('Start', '304'),
              ('Duration', '3'),
              ('SignalLocation', 'SaO2')]),
 OrderedDict([('EventType', 'Arousals|Arousals'),
              ('EventConcept', 'Arousal|Arousal ()'),
              ('Start', '964.8'),
              ('Duration', '25.4'),
              ('SignalLocation', 'EMG')]),
 OrderedDict([('EventType', 'Arousals|Arousals'),
              ('EventConcept', 'Arousal|Arousal ()'),
              ('Start', '1552.2'),
              ('Duration', '23'),
              ('SignalLocation', 'EMG')]),
 OrderedDict([('EventType', 'Arousals|Arousals'),
              ('EventConcept', 'Arousal|Arousal ()'),
              ('Start',

In [24]:
stages_list = list()
for element in scored_events:
    if element['EventType'] == 'Stages|Stages':
        stages_list.append(element)

stages_list

[OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Wake|0'),
              ('Start', '0.0'),
              ('Duration', '840.0')]),
 OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Stage 1 sleep|1'),
              ('Start', '840.0'),
              ('Duration', '30.0')]),
 OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Stage 2 sleep|2'),
              ('Start', '870.0'),
              ('Duration', '90.0')]),
 OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Wake|0'),
              ('Start', '960.0'),
              ('Duration', '30.0')]),
 OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Stage 1 sleep|1'),
              ('Start', '990.0'),
              ('Duration', '30.0')]),
 OrderedDict([('EventType', 'Stages|Stages'),
              ('EventConcept', 'Stage 2 sleep|2'),
              ('Start', '1020.0'),
              ('Duration', '270.0')]),
 Or

In [25]:
phases = list()
for element in stages_list:
    phases.append(element['EventConcept'])
mylist = list(dict.fromkeys(phases))
print(mylist) 

['Wake|0', 'Stage 1 sleep|1', 'Stage 2 sleep|2', 'Stage 3 sleep|3', 'REM sleep|5']


In [29]:
test_phase = stages_list[2]
duration_in_seconds = float(test_phase['Duration'])
duration_in_epochs = int(duration_in_seconds/30)

print(str(duration_in_seconds))
print(str(duration_in_epochs))

90.0
3


In [31]:
start = float(test_phase['Start'])
start * 3000

2610000.0